In [16]:
import numpy as np
import pandas as pd
import random
from openpyxl import load_workbook

In [17]:
wb = load_workbook(filename= 'Dataset.xlsx')
sheet_ranges = wb['Sheet 1']

df = pd.DataFrame(sheet_ranges.values)
df

col = df[1:72][[4,5,6,7,9,10,11,12,13]]
col.columns = ['UAW','S','A','C','UUCW','TCF','ECF','Real','Real_Effort']
col1 = pd.DataFrame(col)

col1.insert(9,"Desired_UUCW", 0 , True)
col1

,UAW,S,A,C,UUCW,TCF,ECF,Real,Real_Effort,Desired_UUCW
1,9,6,10,15,355,0.81,0.84,398.5,7970,0
2,8,4,20,15,445,0.99,0.99,398.1,7962,0
3,9,1,5,20,355,1.03,0.8,396.75,7935,0
4,8,5,10,15,350,0.9,0.91,390.25,7805,0
5,8,1,10,16,345,0.9,0.91,387.9,7758,0
...,...,...,...,...,...,...,...,...,...,...
67,18,2,18,18,460,1.05,0.95,290,5800,0
68,17,1,23,22,565,1.03,0.8,289.55,5791,0
69,19,5,30,10,475,0.95,0.92,289.1,5782,0
70,6,5,15,5,250,1,0.92,288.9,5778,0


In [18]:
for index, row in col1.iterrows():
    UAW = col1.loc[index,"UAW"]
    UUCW = col1.loc[index,"UUCW"]
    TCF = col1.loc[index,"TCF"]
    ECF = col1.loc[index,"ECF"]
    Real_E = col1.loc[index,"Real_Effort"]
    
    desired_UUCW = Real_E/((TCF*ECF)*20)-UAW
    #col1.at(5,"UUCW Algen", desired_UUCW , True)
    col1.at[index, 'Desired_UUCW'] = np.round(desired_UUCW,1)


col1

,UAW,S,A,C,UUCW,TCF,ECF,Real,Real_Effort,Desired_UUCW
1,9,6,10,15,355,0.81,0.84,398.5,7970,576
2,8,4,20,15,445,0.99,0.99,398.1,7962,398
3,9,1,5,20,355,1.03,0.8,396.75,7935,472
4,8,5,10,15,350,0.9,0.91,390.25,7805,468
5,8,1,10,16,345,0.9,0.91,387.9,7758,465
...,...,...,...,...,...,...,...,...,...,...
67,18,2,18,18,460,1.05,0.95,290,5800,272
68,17,1,23,22,565,1.03,0.8,289.55,5791,334
69,19,5,30,10,475,0.95,0.92,289.1,5782,311
70,6,5,15,5,250,1,0.92,288.9,5778,308


In [38]:
array_aePerulangan = []
for index, row in col1.iterrows():
    print()
    print("____ DATA ",index,"____")

    #Setting Parameter

    c1 = 2
    c2 = 2
    r1 = (0, 1)
    r2 = (0, 1)
    iteration_max = 10 #40
    particles = 10 #70
    fitness_value = 200
    productivity = 20
    inertia_max = 0.9
    inertia_min = 0.4
    inertia = 0
    temp = 0
    ae = 0
    particles_position = [0]

    #perhitungan UCP
    def ucp(xSimple, simpleUC, xAverage, averageUC, xComplex, complexUC, uaw, tcf, ecf):
        ucSimple = xSimple * simpleUC
        ucAverage = xAverage * averageUC
        ucComplex = xComplex * complexUC
    
        uucw = ucSimple + ucAverage + ucComplex
        uucp = uaw + uucw
        return uucp * tcf * ecf

    #Generate random Simple Use Case Complexity Weight Parameter
    def randomSimpleUCW(lower_bound=5, upper_bound=7.49, n_particles=10):
        for i in range(n_particles):
            particles_position[i] = random.uniform(lower_bound, upper_bound)
            return particles_position[i]

    #Generate random Average Use Case Complexity Weight Parameter 
    def randomAverageUCW(lower_bound=7.5, upper_bound=12.49, n_particles=10):
        for i in range(n_particles):
            particles_position[i] = random.uniform(lower_bound, upper_bound)
            return particles_position[i]

    #Generate random Complex Use Case Complexity Weight Parameter
    def randomComplexUCW(lower_bound=12.5, upper_bound=15, n_particles=10):
        #inisialisasi posisi awal
        for i in range(n_particles):
            particles_position[i] = random.uniform(lower_bound, upper_bound)
            return particles_position[i]
    
    swarm_size = 5 #70 
    simpleUC = col1.loc[index,"S"]
    averageUC = col1.loc[index,"A"]
    complexUC = col1.loc[index,"C"]
    estimated_effort = 0
    uaw = col1.loc[index,"UAW"]
    uucw = col1.loc[index,"UUCW"]
    tcf = col1.loc[index,"TCF"]
    ecf = col1.loc[index,"ECF"]
    real_effort = col1.loc[index,"Real_Effort"]
    array_aeParticle = []
    ae_minimum = 5000
    particle = []
    
    for i in range(iteration_max):
        inertia = inertia_max - ((inertia_max-inertia_min)*i/iteration_max)
        print('Perulangan ',i)
        print('inertia :', inertia)
        r1 = random.uniform(0, 1)
        r2 = random.uniform(0, 1)
        print('r1 :', r1)
        print('r2 :', r2)
        print()
        
        if i == 0:
            for j in range(swarm_size):    
                xSimple = randomSimpleUCW()
                xAverage = randomAverageUCW()
                xComplex = randomComplexUCW()
                case = ucp(xSimple, simpleUC, xAverage, averageUC, xComplex, complexUC, uaw, tcf, ecf)
                estimated_effort = case * 20
        
                #Selisih estimasi effort
                ae = abs(estimated_effort - real_effort)
            
                #Menyimpan ae minimum dan letaknya
                if ae_minimum > ae:
                    ae_minimum = ae
                    letak_i = i
                    letak_j = j
                    xSimpleMin = xSimple
                    xAverageMin = xAverage
                    xComplexMin = xComplex
            
                #Memasukkan selisih
                #array_aeParticle.append(ae)
                #aeParticle_min = np.min(array_aeParticle)
                #index_of_minimum = np.where(array_aeParticle == aeParticle_min)
        
                #menjumlah 10 angka selisih
                temp = np.array(temp+ae)
         
                #print('Partikel ke', j,':',estimated_effort)
                #print('ucp   ', case)
                #print('ae ', j, ae)
                #print('S     ', xSimple)
                #print('A     ', xAverage)
                #print('C     ', xComplex)
                #print()
            
                #print('estimated effort', estimated_effort)
            
                if j == 0:
                    pbest_simple = xSimple
                    pbest_average = xAverage
                    pbest_complex = xComplex
                
                    vSimple = random.uniform(0, 1)
                    vAverage = random.uniform(0, 1)
                    vComplex = random.uniform(0, 1)
                
                    xSimple = pbest_simple
                    xAveragepbest_average = pbest_average
                    xComplex = pbest_complex
                
                    xAverage = randomAverageUCW()
                    xComplex = randomComplexUCW()
                    case = ucp(xSimple, simpleUC, xAverage, averageUC, xComplex, complexUC, uaw, tcf, ecf)
                    estimated_effort = case * 20
                    
                    #Selisih estimasi effort
                    ae = abs(estimated_effort - real_effort)
                    
                    particle = {"xSimple": xSimple, "xAverage": xAverage, "xComplex": xComplex, "vSimple": vSimple, "vAverage": vAverage, "vComplex": vComplex, "ae": ae}
                    print(particle)
                      
            
                #Hitung velocity
                #vSimple = inertia * vSimple + (c1 * r1) * (pbest_simple - xSimple) + (c2* r2) * 
                
            
            #
            
            #Menyimpan ae minimum dan letaknya
            if ae_minimum > ae:
                ae_minimum = ae
                letak_i = i
                letak_j = j
                xSimpleMin = xSimple
                xAverageMin = xAverage
                xComplexMin = xComplex
            
            #Memasukkan selisih
            #array_aeParticle.append(ae)
            #aeParticle_min = np.min(array_aeParticle)
            #index_of_minimum = np.where(array_aeParticle == aeParticle_min)
        
            #menjumlah 10 angka selisih
            #temp = np.array(temp+ae)
         
            #print('Partikel ke', j,':',estimated_effort)
            #print('ucp   ', case)
            #print('ae ', j, ae)
            #print('S     ', xSimple)
            #print('A     ', xAverage)
            #print('C     ', xComplex)
            #print()
            
            #print('estimated effort', estimated_effort)
            
        #print('ae terkecil : ',np.min(aeParticle_min))
        #print('letak ae : ',index_of_minimum)
        #print('pada perulangan ke :', aeParticle_min.index(min(aeParticle_min)))
        #mae = temp/swarm_size
        #print('MAE :', mae)
        print()
        #array_aePerulangan.append(aeParticle_min)
    print('ae_minimum', ae_minimum)
    print('i', letak_i)
    print('j', letak_j)
    print('S Min', xSimpleMin)
    print('A Min', xAverageMin)
    print('C Min', xComplexMin)
    #print('ae terkecil keseluruhan : ',np.min(array_aePerulangan))
    #print('letak ae : ',index_of_minimum)


____ DATA  1 ____
Perulangan  0
inertia : 0.9
r1 : 0.2063437421217983
r2 : 0.401058907002282

{'xSimple': 5.764205418065992, 'xAverage': 8.924731339868593, 'xComplex': 13.667840802967861, 'vSimple': 0.59308284760697, 'vAverage': 0.9960287921418597, 'vComplex': 0.38559895838305247, 'ae': 3372.53505059463}

Perulangan  1
inertia : 0.85
r1 : 0.558878956191332
r2 : 0.6733188118632989


Perulangan  2
inertia : 0.8
r1 : 0.1954508621531138
r2 : 0.7284518599898238


Perulangan  3
inertia : 0.75
r1 : 0.7557840181007135
r2 : 0.10749486406714348


Perulangan  4
inertia : 0.7
r1 : 0.9157570075509629
r2 : 0.9116511799098828


Perulangan  5
inertia : 0.65
r1 : 0.8700750736404597
r2 : 0.8016985721884606


Perulangan  6
inertia : 0.6000000000000001
r1 : 0.4526040383842904
r2 : 0.08868258985548971


Perulangan  7
inertia : 0.55
r1 : 0.8245970429643539
r2 : 0.6583271474956409


Perulangan  8
inertia : 0.5
r1 : 0.016565618083386746
r2 : 0.914622720285409


Perulangan  9
inertia : 0.45
r1 : 0.28819691163

A Min 11.667974047524254
C Min 13.0016811983188

____ DATA  17 ____
Perulangan  0
inertia : 0.9
r1 : 0.6301818361431856
r2 : 0.9008551735481176

{'xSimple': 7.408001544632796, 'xAverage': 10.979407538167218, 'xComplex': 14.132255887054122, 'vSimple': 0.5452736954782885, 'vAverage': 0.2157071178478278, 'vComplex': 0.6901944026302869, 'ae': 2152.627994191126}

Perulangan  1
inertia : 0.85
r1 : 0.19609394415466563
r2 : 0.964331693308553


Perulangan  2
inertia : 0.8
r1 : 0.37373889605238864
r2 : 0.1207484838821492


Perulangan  3
inertia : 0.75
r1 : 0.7918858834940888
r2 : 0.48342841393169667


Perulangan  4
inertia : 0.7
r1 : 0.14862078670359358
r2 : 0.655473279431257


Perulangan  5
inertia : 0.65
r1 : 0.8619669598774329
r2 : 0.5093502407077943


Perulangan  6
inertia : 0.6000000000000001
r1 : 0.8092648173115887
r2 : 0.8700349419457956


Perulangan  7
inertia : 0.55
r1 : 0.26076685751955386
r2 : 0.27408334133950696


Perulangan  8
inertia : 0.5
r1 : 0.9140946994196842
r2 : 0.44671074380

inertia : 0.45
r1 : 0.7309978021554154
r2 : 0.717525756205983


ae_minimum 4220.4456136843255
i 0
j 3
S Min 5.757637296718039
A Min 11.431504384394175
C Min 13.686591045800919

____ DATA  28 ____
Perulangan  0
inertia : 0.9
r1 : 0.5302083025408934
r2 : 0.13977382116528858

{'xSimple': 6.2679460734163115, 'xAverage': 10.38340552262787, 'xComplex': 14.129330742781109, 'vSimple': 0.7633705268448038, 'vAverage': 0.774231771781079, 'vComplex': 0.6980620483344382, 'ae': 1699.0699102407225}

Perulangan  1
inertia : 0.85
r1 : 0.4342324665204199
r2 : 0.5418669025485807


Perulangan  2
inertia : 0.8
r1 : 0.3851434566157137
r2 : 0.6160482772237347


Perulangan  3
inertia : 0.75
r1 : 0.4060698626555357
r2 : 0.7194956021977855


Perulangan  4
inertia : 0.7
r1 : 0.013925921605894787
r2 : 0.6050525761868771


Perulangan  5
inertia : 0.65
r1 : 0.14763929568593404
r2 : 0.3743555019672643


Perulangan  6
inertia : 0.6000000000000001
r1 : 0.5248803567812043
r2 : 0.37615184755874065


Perulangan  7
inerti


Perulangan  9
inertia : 0.45
r1 : 0.3057717866197237
r2 : 0.16301498497580447


ae_minimum 1051.05084003288
i 0
j 3
S Min 5.728056586677102
A Min 12.44079474363104
C Min 13.766522546980367

____ DATA  43 ____
Perulangan  0
inertia : 0.9
r1 : 0.640156743599422
r2 : 0.5168121787968164

{'xSimple': 5.126681874793799, 'xAverage': 11.608523687920325, 'xComplex': 12.999770792073619, 'vSimple': 0.7334657009219797, 'vAverage': 0.4381292377109398, 'vComplex': 0.3137809053403251, 'ae': 101.49876890512678}

Perulangan  1
inertia : 0.85
r1 : 0.3552244939964764
r2 : 0.6518277536069037


Perulangan  2
inertia : 0.8
r1 : 0.5785802688309213
r2 : 0.02847612591736237


Perulangan  3
inertia : 0.75
r1 : 0.3040445565064098
r2 : 0.29074596047777235


Perulangan  4
inertia : 0.7
r1 : 0.5481839656688411
r2 : 0.3649484503449628


Perulangan  5
inertia : 0.65
r1 : 0.5511020373475894
r2 : 0.5401144959791502


Perulangan  6
inertia : 0.6000000000000001
r1 : 0.9348669902476165
r2 : 0.1323410399408399


Perulanga

r2 : 0.8310572261294656


Perulangan  7
inertia : 0.55
r1 : 0.3965657131733109
r2 : 0.10521746634909879


Perulangan  8
inertia : 0.5
r1 : 0.0807860528413551
r2 : 0.4737693607567023


Perulangan  9
inertia : 0.45
r1 : 0.17640921497589035
r2 : 0.3332310526109715


ae_minimum 759.4024957393976
i 0
j 0
S Min 5.109078793276806
A Min 9.023273357090105
C Min 12.501342725054052

____ DATA  56 ____
Perulangan  0
inertia : 0.9
r1 : 0.36587382140455516
r2 : 0.2541846736244552

{'xSimple': 6.0454500208632584, 'xAverage': 11.020593142795384, 'xComplex': 14.162821574821924, 'vSimple': 0.477628559358137, 'vAverage': 0.8288312603826632, 'vComplex': 0.3028601787380819, 'ae': 3125.7377121527643}

Perulangan  1
inertia : 0.85
r1 : 0.7901727322225531
r2 : 0.2631013576623429


Perulangan  2
inertia : 0.8
r1 : 0.1891595365078429
r2 : 0.8007448516396987


Perulangan  3
inertia : 0.75
r1 : 0.8676278420120377
r2 : 0.043684374197093034


Perulangan  4
inertia : 0.7
r1 : 0.7921000473076434
r2 : 0.81237702402186

r1 : 0.6550253399938771
r2 : 0.749236853242158


Perulangan  7
inertia : 0.55
r1 : 0.09941506612595252
r2 : 0.5507691268974986


Perulangan  8
inertia : 0.5
r1 : 0.35672803931322705
r2 : 0.8763681012401301


Perulangan  9
inertia : 0.45
r1 : 0.3000216780093298
r2 : 0.902639808599765


ae_minimum 2289.6540276947962
i 0
j 0
S Min 7.414666260699244
A Min 8.925855700825288
C Min 13.791610223094402

____ DATA  70 ____
Perulangan  0
inertia : 0.9
r1 : 0.3340539030271449
r2 : 0.1545246748155532

{'xSimple': 6.166301154733587, 'xAverage': 11.474804699931163, 'xComplex': 14.132757509220156, 'vSimple': 0.20990654446609258, 'vAverage': 0.423911704656855, 'vComplex': 0.3544947403307025, 'ae': 633.0405057352546}

Perulangan  1
inertia : 0.85
r1 : 0.1823076835284947
r2 : 0.4613667365968662


Perulangan  2
inertia : 0.8
r1 : 0.7756732215271382
r2 : 0.3410189803089445


Perulangan  3
inertia : 0.75
r1 : 0.381227569969397
r2 : 0.8476144436996214


Perulangan  4
inertia : 0.7
r1 : 0.8331014701276567
r2 